In [1]:
import numpy as np
from numpy import linalg as LA

    
def plsr(X,Y,factors,thre,T,U,P,Q,W,B):
    rows=len(X)
    colsX=len(X[0])
    colsY=len(Y[0])
    E,F=X.copy(),Y.copy()
    
    for factor in range(factors):
        t=E[:,0]
        u=F[:,0]
        w=np.ones((colsX))
        q=np.ones((colsY))
        t_norm=LA.norm(t)
        
        while t_norm>1e-7:
            
            temp=t.copy()
            w=u.T@E/(u@u.T)
            w=w/LA.norm(w)
            t=E@w.T/(w@w.T)
            q=t@F/(t@t.T)
            q=q/LA.norm(q)
            u=F@q.T/(q@q.T)
            t_norm=LA.norm(t-temp)
        
        p=t@E/(t@t.T)
        t=t*LA.norm(p)
        w=w*LA.norm(p)
        p=p/LA.norm(p)
        
        b=u@t.T/(t@t.T)
        F_=F.copy()
        for i in range(len(t)):
            for j in range(len(p)):
                E[i,j]-=t[i]*p[j]
            for j in range(len(q)):
                F_[i,j]-=b*t[i]*q[j]
        T[:,factor]=t
        U[:,factor]=u
        P[:,factor]=p
        Q[:,factor]=q
        W[:,factor]=w
        B[factor]=b
        
        if LA.norm(F_)<thre or abs(LA.norm(F_)-LA.norm(F))<1e-10:
            return #already converged
        F=F_

def predict(W,Q,P,X,Y_,B,factors):
    
    for i in range(factors):
        t=X@W[:,i]
        for k in range(len(t)):
            for j in range(len(P)):
                X[k,j]-=t[k]*P[j,i]
            for j in range(len(Q)):
                Y_[k,j]+=B[i]*t[k]*Q[j,i]
    return Y_
    


In [2]:
X = np.array([[0., 0., 1.], [1.,0.,0.], [2.,2.,2.], [2.,5.,4.]])
Y = np.array([[0.1, -0.2], [0.9, 1.1], [6.2, 5.9], [11.9, 12.3]])
rows=len(X)
colsX=len(X[0])
colsY=len(Y[0])
factors=2
T=np.zeros((rows,factors))
U=np.zeros((rows,factors))
P=np.zeros((colsX,factors))
Q=np.zeros((colsY,factors))
W=np.zeros((colsX,colsX))
B=np.zeros((colsX))
Y_=np.zeros((rows,colsY))
plsr(X,Y,factors,1e-7,T,U,P,Q,W,B)


In [3]:
predict(W,Q,P,X,Y_,B,factors)

array([[ 0.28197796,  0.11093105],
       [ 1.15536022,  1.28576758],
       [ 6.2392634 ,  6.39321506],
       [11.85009987, 12.01480385]])

In [4]:
import numba
import numpy as np
from numpy import linalg as LA
from numba import jit
from numba import int32, int64, float32, float64
from numpy.testing import assert_array_almost_equal

@jit(nopython=True)
def plsr_numba(X,Y,factors,thre,T,U,P,Q,W,B):
    rows=len(X)
    colsX=len(X[0])
    colsY=len(Y[0])
    E,F=X.copy(),Y.copy()
    
    for factor in range(factors):
        t=E[:,0]
        u=F[:,0]
        w=np.ones((colsX))
        q=np.ones((colsY))
        t_norm=LA.norm(t)
        
        while t_norm>1e-7:
            temp=t.copy()
            w=u.T@E/(u@u.T)
            w=w/LA.norm(w)
            t=E@w.T/(w@w.T)
            q=t@F/(t@t.T)
            q=q/LA.norm(q)
            u=F@q.T/(q@q.T)
            t_norm=LA.norm(t-temp)
            
            
        p=t@E/(t@t.T)
        t=t*LA.norm(p)
        w=w*LA.norm(p)
        p=p/LA.norm(p)
        
        b=u@t.T/(t@t.T)
    
#         E=E-t.reshape(1,-1).T@p.reshape(1,-1)
#         F_=F-b*t.reshape(1,-1).T@q.reshape(1,-1)  
        F_=F.copy()
        for i in range(len(t)):
            for j in range(len(p)):
                E[i,j]-=t[i]*p[j]
            for j in range(len(q)):
                F_[i,j]-=b*t[i]*q[j]
            
        T[:,factor]=t
        U[:,factor]=u
        P[:,factor]=p
        Q[:,factor]=q
        W[:,factor]=w
        B[factor]=b
        
        if LA.norm(F_)<thre or abs(LA.norm(F_)-LA.norm(F))<1e-10:
            return #already converged
        F=F_
        
@jit(nopython=True)
def predict_numba(W,Q,P,X,Y_,B,factors):
    
    for i in range(factors):
        t=X@W[:,i]
        for k in range(len(t)):
            for j in range(len(P)):
                X[k,j]-=t[k]*P[j,i]
            for j in range(len(Q)):
                Y_[k,j]+=B[i]*t[k]*Q[j,i]
    return Y_

In [5]:
X = np.array([[0., 0., 1.], [1.,0.,0.], [2.,2.,2.], [2.,5.,4.]])
Y = np.array([[0.1, -0.2], [0.9, 1.1], [6.2, 5.9], [11.9, 12.3]])
rows=len(X)
colsX=len(X[0])
colsY=len(Y[0])
factors=2
T=np.zeros((rows,factors))
U=np.zeros((rows,factors))
P=np.zeros((colsX,factors))
Q=np.zeros((colsY,factors))
W=np.zeros((colsX,colsX))
B=np.zeros((colsX))
Y_=np.zeros((rows,colsY))
%timeit -r3 -n3 plsr_numba(X,Y,factors,1e-7,T,U,P,Q,W,B)


<ipython-input-4-8e7842a62633>:24: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  w=u.T@E/(u@u.T)
<ipython-input-4-8e7842a62633>:24: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 2d, C))
  w=u.T@E/(u@u.T)
<ipython-input-4-8e7842a62633>:27: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  q=t@F/(t@t.T)
<ipython-input-4-8e7842a62633>:27: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 2d, C))
  q=t@F/(t@t.T)
<ipython-input-4-8e7842a62633>:33: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 1d, A))
  p=t@E/(t@t.T)
<ipython-input-4-8e7842a62633>:33: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 2d, C)

The slowest run took 24282.99 times longer than the fastest. This could mean that an intermediate result is being cached.
405 ms ± 573 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [6]:
%timeit -r3 -n3 predict_numba(W,Q,P,X,Y_,B,factors)

<ipython-input-4-8e7842a62633>:64: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 1d, A))
  t=X@W[:,i]
/opt/conda/lib/python3.6/site-packages/numba/typing/npydecl.py:958: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (array(float64, 2d, C), array(float64, 1d, A))
  warnings.warn(NumbaPerformanceWarning(msg))


The slowest run took 40862.96 times longer than the fastest. This could mean that an intermediate result is being cached.
38.1 ms ± 53.9 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [7]:
X = np.array([[0., 0., 1.], [1.,0.,0.], [2.,2.,2.], [2.,5.,4.]])
Y = np.array([[0.1, -0.2], [0.9, 1.1], [6.2, 5.9], [11.9, 12.3]])
rows=len(X)
colsX=len(X[0])
colsY=len(Y[0])
factors=2
T=np.zeros((rows,factors))
U=np.zeros((rows,factors))
P=np.zeros((colsX,factors))
Q=np.zeros((colsY,factors))
W=np.zeros((colsX,colsX))
B=np.zeros((colsX))
Y_=np.zeros((rows,colsY))
%timeit -r3 -n3 plsr(X,Y,factors,1e-7,T,U,P,Q,W,B)

X = np.array([[0., 0., 1.], [1.,0.,0.], [2.,2.,2.], [2.,5.,4.]])
Y = np.array([[0.1, -0.2], [0.9, 1.1], [6.2, 5.9], [11.9, 12.3]])
rows=len(X)
colsX=len(X[0])
colsY=len(Y[0])
factors=2
T=np.zeros((rows,factors))
U=np.zeros((rows,factors))
P=np.zeros((colsX,factors))
Q=np.zeros((colsY,factors))
W=np.zeros((colsX,colsX))
B=np.zeros((colsX))
Y_=np.zeros((rows,colsY))
%timeit -r3 -n3 plsr_numba(X,Y,factors,1e-7,T,U,P,Q,W,B)

727 µs ± 133 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)
59.8 µs ± 7.85 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [8]:
assert_array_almost_equal(predict_numba(W,Q,P,X,Y_,B,factors), predict(W,Q,P,X,Y_,B,factors))
%timeit -r3 -n3 predict(W,Q,P,X,Y_,B,factors)
%timeit -r3 -n3 predict_numba(W,Q,P,X,Y_,B,factors)


53 µs ± 3.23 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)
The slowest run took 5.00 times longer than the fastest. This could mean that an intermediate result is being cached.
5.48 µs ± 4.38 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [9]:
%reload_ext cython

In [10]:
%%cython -a

import cython


import numpy as np
from libc.math cimport sqrt
from cpython cimport array
# import array

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def plsr_cython(double[:,:]X,double[:,:]Y,int factors,double thre,double[:,:] T,double[:,:] U,double[:,:] P,double[:,:] Q,double[:,:] W,double[:] B):
    
    cdef int rows,colsX,colsY
    cdef double b,sum_,t_norm,norm
    cdef double[:] t,u,w,q,p,temp
    cdef double[:,:] E,F
    rows=X.shape[0]
    colsX=X.shape[1]
    colsY=Y.shape[1]
    E,F=X.copy(),Y.copy()
    
    for factor in range(factors):
        t=E.copy()[:,0]
        u=F.copy()[:,0]
        w=np.zeros(colsX)#array.array('d', [0,0,0])
        q=np.zeros(colsY)#array.array('d', [0,0])
        p=np.zeros(colsX)#array.array('d', [0,0,0])
        sum_=0
        for i in range(rows):
            sum_+=t[i]*t[i]
        t_norm=sqrt(sum_)
        
        while t_norm>1e-7:      
            temp=t.copy()
            for j in range(colsX):
                w[j]=0
                for i in range(rows):
                    w[j]+=E[i,j]*u[i]

            sum_=0
            for i in range(rows):
                sum_+=u[i]*u[i]

            for i in range(colsX):
                w[i]=w[i]/sum_

            sum_=0
            for i in range(colsX):
                sum_+=w[i]*w[i]
            norm=sqrt(sum_)
            
            for i in range(colsX):
                w[i]=w[i]/norm

            for i in range(rows):
                t[i]=0
                for j in range(colsX):
                    t[i]+=E[i,j]*w[j]

            sum_=0
            for i in range(colsX):
                sum_+=w[i]*w[i]  

            for i in range(rows):
                t[i]=t[i]/sum_
 
            for j in range(colsY):
                q[j]=0
                for i in range(rows):
                    q[j]+=F[i,j]*t[i]
            sum_=0
            for i in range(rows):
                sum_+=t[i]*t[i]  
            for i in range(colsY):
                q[i]=q[i]/sum_
            
            sum_=0
            for i in range(colsY):
                sum_+=q[i]*q[i]
            norm=sqrt(sum_)

            for i in range(colsY):
                q[i]=q[i]/norm
            

            for i in range(rows):
                u[i]=0
                for j in range(colsY):
                    u[i]+=F[i,j]*q[j]
            
            sum_=0
            for i in range(colsY):
                sum_+=q[i]*q[i]  

            for i in range(rows):
                u[i]=u[i]/sum_

            for i in range(rows):
                temp[i]-=t[i]
            
            sum_=0
            for i in range(rows):
                sum_+=temp[i]*temp[i]
            t_norm=sqrt(sum_)
            
        for j in range(colsX):
            p[j]=0
            for i in range(rows):
                p[j]+=E[i,j]*t[i]
                
        sum_=0
        for i in range(rows):
            sum_+=t[i]*t[i]  
        for i in range(colsX):
            p[i]=p[i]/sum_

        b=0
        for i in range(rows):
            b+=u[i]*t[i]/sum_
        sum_=0
        for i in range(colsX):
            sum_+=p[i]*p[i]
        norm=sqrt(sum_)
        
        for i in range(rows):
            t[i]=t[i]*norm
        for i in range(rows):
            w[i]=w[i]*norm
        for i in range(rows):
            p[i]=p[i]/norm

        for i in range(rows):
            for j in range(colsX):
                E[i,j]-=t[i]*p[j]
            for j in range(colsY):
                F[i,j]-=b*t[i]*q[j]

        T[:,factor]=t
        U[:,factor]=u
        P[:,factor]=p
        Q[:,factor]=q
        W[:,factor]=w
        B[factor]=b
        
        
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def predict_cython(double[:,:]W,double[:,:]Q,double[:,:]P,double[:,:]X,double[:,:]Y_,double[:]B,int factors):
    cdef double[:] t
    cdef int rows,colsX,colsY
    rows=X.shape[0]
    colsX=X.shape[1]
    colsY=Y_.shape[1]
    for i in range(factors):

        t=array.array('d', [0,0,0,0])
        for k in range(rows):
            for j in range(colsX):
                t[k]+=X[k,j]*W[j,i]
        for k in range(rows):
            for j in range(colsX):
                X[k,j]-=t[k]*P[j,i]
            for j in range(colsY):
                Y_[k,j]+=B[i]*t[k]*Q[j,i]

    return Y_
    

In [11]:
X = np.array([[0., 0., 1.], [1.,0.,0.], [2.,2.,2.], [2.,5.,4.]])
Y = np.array([[0.1, -0.2], [0.9, 1.1], [6.2, 5.9], [11.9, 12.3]])
rows=len(X)
colsX=len(X[0])
colsY=len(Y[0])
factors=2
T=np.zeros((rows,factors))
U=np.zeros((rows,factors))
P=np.zeros((colsX,factors))
Q=np.zeros((colsY,factors))
W=np.zeros((colsX,colsX))
B=np.zeros((colsX))
Y_=np.zeros((rows,colsY))
plsr_cython(X,Y,factors,1e-7,T,U,P,Q,W,B)


In [47]:
assert_array_almost_equal(predict_cython(W,Q,P,X,Y_,B,factors), predict(W,Q,P,X,Y_,B,factors))
%timeit -r3 -n3 predict_cython(W,Q,P,X,Y_,B,factors)
%timeit -r3 -n3 predict_numba(W,Q,P,X,Y_,B,factors)

7.21 µs ± 2.11 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)
6.44 µs ± 3.83 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [13]:
Y_=predict_cython(W,Q,P,X,Y_,B,factors)
for i in range(4):
    for j in range(2):
        print(Y_[i][j])

0.17865430410062003
-0.01724689844824463
1.221677076922812
1.368021337117836
6.2732318007394205
6.435281125786279
11.839393966979493
12.001385701560432


In [51]:
X = np.array([[0., 0., 1.], [1.,0.,0.], [2.,2.,2.], [2.,5.,4.]])
Y = np.array([[0.1, -0.2], [0.9, 1.1], [6.2, 5.9], [11.9, 12.3]])
rows=len(X)
colsX=len(X[0])
colsY=len(Y[0])
factors=2
T=np.zeros((rows,factors))
U=np.zeros((rows,factors))
P=np.zeros((colsX,factors))
Q=np.zeros((colsY,factors))
W=np.zeros((colsX,colsX))
B=np.zeros((colsX))
Y_=np.zeros((rows,colsY))
%timeit -r3 -n3 plsr_numba(X,Y,factors,1e-7,T,U,P,Q,W,B)

X = np.array([[0., 0., 1.], [1.,0.,0.], [2.,2.,2.], [2.,5.,4.]])
Y = np.array([[0.1, -0.2], [0.9, 1.1], [6.2, 5.9], [11.9, 12.3]])
rows=len(X)
colsX=len(X[0])
colsY=len(Y[0])
factors=2
T=np.zeros((rows,factors))
U=np.zeros((rows,factors))
P=np.zeros((colsX,factors))
Q=np.zeros((colsY,factors))
W=np.zeros((colsX,colsX))
B=np.zeros((colsX))
Y_=np.zeros((rows,colsY))
%timeit -r3 -n3 plsr_cython(X,Y,factors,1e-7,T,U,P,Q,W,B)



68.1 µs ± 17.2 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)
36 µs ± 6.94 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)
